In [ ]:
from collections.abc import Iterator
from random import randint, choice

N = 8  # Board size


def heuristic(board: str) -> int:
    """The heuristic cost function h is the number of pairs of queens that are attacking each other;
    this will be zero only for solutions. (It counts as an attack if two pieces are in the same line,
    even if there is an intervening piece between them.)
    """
    h = 0

    for i in range(N):
        # Value in column i
        row_i = int(board[i])

        for j in range(i + 1, N):
            # Value in column j
            row_j = int(board[j])

            # Same row
            if row_i == row_j:
                h += 1

            # Same diagonal
            if (j - i) == abs(row_j - row_i):
                h += 1

    return h


def random_state() -> str:
    """Generate a random board state by placing one queen in each column at a random row position."""
    res = ''

    for col in range(N):
        res += str(randint(1, N))

    return res


def successors(current: str) -> list[str]:
    """Generate all the successors of a state, i.e. all possible states generated by moving a single queen
    to another square in the same column (so each state has 8 x 7 = 56 successors)."""
    successors = []

    for col in range(N):
        current_row = int(current[col])

        for new_row in range(1, N + 1):
            if new_row != current_row:
                # Create new board with queen moved
                new_board = current[:col] + str(new_row) + current[col + 1 :]
                successors.append(new_board)

    return successors


def successors_generator(current: str) -> Iterator[str]:
    """Generate successor states lazily using a generator.
    
    Memory-efficient alternative to successors() that yields states one at a time
    instead of creating the entire list in memory. Useful for first-choice hill climbing.
    """
    for col in range(N):
        current_row = int(current[col])

        for new_row in range(1, N + 1):
            if new_row != current_row:
                # Create new board with queen moved
                new_board = current[:col] + str(new_row) + current[col + 1 :]
                yield new_board


def display(board: str) -> None:
    """Display the board state in a visual format."""
    grid = [['_'] * N for _ in range(N)]

    for col in range(N):
        current_row = int(board[col])
        grid[N - current_row][col] = 'X'

    for row in grid:
        print(' '.join(row))

In [ ]:
def hill_climbing(initial_state: str) -> tuple[str, dict[str, int | bool]]:
    """Steepest-ascent hill climbing algorithm."""
    current, h_current = initial_state, heuristic(initial_state)
    steps = 0

    while True:
        children = [
            (successor, heuristic(successor)) for successor in successors(current)
        ]
        neighbor, h_neighbor = min(children, key=lambda x: x[1])
        steps += 1
        if h_neighbor >= h_current:
            stats = {'steps': steps, 'h_final': h_current, 'success': h_current == 0}
            return current, stats
        else:
            current, h_current = neighbor, h_neighbor


def hill_climbing_sideways_move(initial_state: str) -> tuple[str, dict]:
    current, h_current = initial_state, heuristic(initial_state)
    sideways_move = 0
    steps = 0

    while True:
        children = [
            (successor, heuristic(successor)) for successor in successors(current)
        ]
        h_min = min(children, key=lambda x: x[1])[1]
        children = [child for child in children if child[1] == h_min]
        # We choose a random neighbor among the best ones
        neighbor, h_neighbor = choice(children)
        steps += 1
        if (h_neighbor > h_current) or (sideways_move == 100):
            stats = {'steps': steps, 'h_final': h_current, 'success': h_current == 0}
            return current, stats
        elif h_neighbor == h_current:
            sideways_move += 1
            current, h_current = neighbor, h_neighbor
        else:
            current, h_current = neighbor, h_neighbor


def first_choice_hill_climbing(initial_state: str) -> tuple[str, dict]:
    current, h_current = initial_state, heuristic(initial_state)
    steps = 0

    while True:
        for child in successors_generator(current):
            h_child = heuristic(child)

            if h_child < h_current:
                steps += 1
                current, h_current = child, h_child
                break
        # finally found the use of an 'else' on a 'for' loop, fuck yeah!
        else:
            stats = {'steps': steps, 'h_final': h_current, 'success': h_current == 0}
            return current, stats


def random_restart_hill_climbing() -> tuple[str, dict]:
    restarts, total_steps = 0, 0

    while True:
        current_state = random_state()
        res, stats = hill_climbing(current_state)
        total_steps += stats['steps']
        if stats['h_final'] == 0:
            stats['restarts'], stats['total_steps'] = restarts, total_steps
            return res, stats
        restarts += 1


def random_restart_hill_climbing_sideways_move() -> tuple[str, dict]:
    restarts, total_steps = 0, 0

    while True:
        current_state = random_state()
        res, stats = hill_climbing_sideways_move(current_state)
        total_steps += stats['steps']
        if stats['h_final'] == 0:
            stats['restarts'], stats['total_steps'] = restarts, total_steps
            return res, stats
        restarts += 1

In [3]:
s = '43254323'
display(s)
print(s)

_ _ _ _ _ _ _ _
_ _ _ _ _ _ _ _
_ _ _ _ _ _ _ _
_ _ _ X _ _ _ _
X _ _ _ X _ _ _
_ X _ _ _ X _ X
_ _ X _ _ _ X _
_ _ _ _ _ _ _ _
43254323


In [4]:
print('>>> Hill Climbing:')
res, stats = hill_climbing(s)
display(res)
print(res)
stats

>>> Hill Climbing:
_ _ _ _ _ _ _ _
_ _ _ _ _ _ X _
_ X _ _ _ _ _ _
_ _ _ X _ _ _ _
_ _ _ _ _ X _ _
X _ _ _ _ _ _ X
_ _ X _ _ _ _ _
_ _ _ _ X _ _ _
36251473


{'steps': 6, 'h_final': 1, 'success': False}

In [5]:
print('>>> Hill Climbing Sideways Move:')
res, stats = hill_climbing_sideways_move(s)
display(res)
print(res)
stats

>>> Hill Climbing Sideways Move:
_ _ _ _ _ _ X _
_ _ _ _ X _ _ _
_ X _ _ _ _ _ _
_ _ _ _ _ X _ _
X _ _ _ _ _ _ _
_ _ _ X _ _ _ _
_ _ _ _ _ _ _ X
_ _ X _ _ _ _ _
46137582


{'steps': 107, 'h_final': 1, 'success': False}

In [6]:
print('>>> First Choice Hill Climbing:')
res, stats = first_choice_hill_climbing(s)
display(res)
print(res)
stats

>>> First Choice Hill Climbing:
_ X _ _ _ _ _ _
_ _ _ _ _ X _ _
X _ _ _ _ _ _ _
_ _ X _ _ _ _ _
_ _ _ X _ _ _ _
_ _ _ _ _ _ _ X
_ _ _ _ _ _ X _
_ _ _ _ X _ _ _
68541723


{'steps': 11, 'h_final': 2, 'success': False}

In [7]:
### From a random state

In [8]:
r = random_state()
display(r)
print(r)

_ _ _ _ _ X _ _
_ X _ _ _ _ _ _
X _ X _ _ _ X _
_ _ _ _ _ _ _ _
_ _ _ _ _ _ _ _
_ _ _ _ _ _ _ _
_ _ _ _ _ _ _ X
_ _ _ X X _ _ _
67611862


In [9]:
print('>>> Hill Climbing:')
res, stats = hill_climbing(r)
display(res)
print(res)
stats

>>> Hill Climbing:
_ _ _ _ _ X _ _
_ X _ _ _ _ _ _
_ _ _ _ _ _ X _
_ _ _ _ _ _ _ _
_ _ _ _ _ _ _ X
_ _ X _ X _ _ _
X _ _ _ _ _ _ _
_ _ _ X _ _ _ _
27313864


{'steps': 5, 'h_final': 1, 'success': False}

In [10]:
print('>>> Hill Climbing Sideways Move:')
res, stats = hill_climbing_sideways_move(r)
display(res)
print(res)
stats

>>> Hill Climbing Sideways Move:
_ _ _ _ _ X _ _
_ _ X _ _ _ _ _
_ _ _ _ _ _ X _
_ X _ _ _ _ _ _
_ _ _ _ _ _ _ X
_ _ _ _ X _ _ _
X _ _ _ _ _ _ _
_ _ _ X _ _ _ _
25713864


{'steps': 7, 'h_final': 0, 'success': True}

In [11]:
print('>>> First choice Hill Climbing:')
res, stats = first_choice_hill_climbing(r)
display(res)
print(res)
stats

>>> First choice Hill Climbing:
_ _ _ _ _ X _ _
_ X _ _ _ _ _ _
_ _ _ _ _ _ X _
_ _ _ _ _ _ _ _
_ _ _ _ _ _ _ X
_ _ X _ X _ _ _
X _ _ _ _ _ _ _
_ _ _ X _ _ _ _
27313864


{'steps': 5, 'h_final': 1, 'success': False}

In [12]:
# %timeit hill_climbing(r)
# %timeit hill_climbing_sideways_move(r)
# %timeit first_choice_hill_climbing(r)

In [13]:
### Random restart algorithms

In [14]:
print('>>> Random restart Hill Climbing:')
res, stats = random_restart_hill_climbing()
display(res)
print(res)
stats

>>> Random restart Hill Climbing:
_ X _ _ _ _ _ _
_ _ _ _ _ _ X _
_ _ _ _ X _ _ _
_ _ _ _ _ _ _ X
X _ _ _ _ _ _ _
_ _ _ X _ _ _ _
_ _ _ _ _ X _ _
_ _ X _ _ _ _ _
48136275


{'steps': 5, 'h_final': 0, 'success': True, 'restarts': 17, 'total_steps': 74}

In [15]:
print('>>> Random restart Hill Climbing Sideways Move:')
res, stats = random_restart_hill_climbing_sideways_move()
display(res)
print(res)
stats

>>> Random restart Hill Climbing Sideways Move:
X _ _ _ _ _ _ _
_ _ _ _ _ X _ _
_ _ _ _ _ _ _ X
_ _ X _ _ _ _ _
_ _ _ _ _ _ X _
_ _ _ X _ _ _ _
_ X _ _ _ _ _ _
_ _ _ _ X _ _ _
82531746


{'steps': 17, 'h_final': 0, 'success': True, 'restarts': 0, 'total_steps': 17}

In [16]:
# res = set()
# while len(res) < 92:
#     res.add(random_restart_hill_climbing())
# len(res)
# # res

In [17]:
# res = set()
# while len(res) < 92:
#     res.add(random_restart_hill_climbing_sideways_move())
# len(res)
# # res